<a href="https://colab.research.google.com/github/vaniamv/final-project-edit/blob/main/full_streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Aplication in Real Time to Read Carris API - group 1 - ETL approach

This notebook documents the steps to implement a data pipeline leveraging Google Cloud Platform (GCP), following an ETL (Extract, Transform, Load) approach. The pipeline processes data in three stages:

Streaming Ingestion and Transformation (Extract and Transform):
Data is ingested in real-time from a bucket that gets the vehicles endpoint of  Carris API. During ingestion, transformations are applied directly to the data stream, such as cleaning, enrichment, and standardization, ensuring that only processed and structured data flows through the pipeline.

Loading Transformed Data:
The pre-processed data is then stored in a silver layer bucket on GCP. This layer serves as a structured repository, optimized for downstream analytical queries and consumption.

By prioritizing the ETL approach, this pipeline ensures that the data is transformed as it is ingested, minimizing the need for post-processing and enabling faster delivery of structured and actionable insights.

---



---



1. Authentication to Google Cloud Platform (GCP)



In [ ]:
# autentication to gcloud with login

!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=bkX1TEEMxAk1ufk0TNMXLpxTBzMkvh&prompt=consent&token_usage=remote&access_type=offline&code_challenge=Th6LJRA5MLhPhEsNsIjFgdVCpHXQVdLv9MzdbZcQTCU&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0ASVgi3JmNVNq5TW3c2Urfe3hULxlpQj8BuzpruL0OpAeiyGFxDPccJhNjLv--IhlCKhZxA

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [ ]:
# download connector and save it local

!wget https://repo1.maven.org/maven2/com/google/cloud/bigdataoss/gcs-connector/hadoop3-2.2.7/gcs-connector-hadoop3-2.2.7-shaded.jar -P /usr/local/lib/

--2025-01-22 13:58:52--  https://repo1.maven.org/maven2/com/google/cloud/bigdataoss/gcs-connector/hadoop3-2.2.7/gcs-connector-hadoop3-2.2.7-shaded.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 2a04:4e42:4c::209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33831577 (32M) [application/java-archive]
Saving to: ‘/usr/local/lib/gcs-connector-hadoop3-2.2.7-shaded.jar’

gcs-connector-hadoo 100%[===================>]  32.26M   172MB/s    in 0.2s    

2025-01-22 13:58:52 (172 MB/s) - ‘/usr/local/lib/gcs-connector-hadoop3-2.2.7-shaded.jar’ saved [33831577/33831577]





---


---
2. Initialize SparkSession and set up the access to GSC

In [ ]:
# import libraries

import os
from pyspark.sql import SparkSession

#spark session
spark = SparkSession.builder \
    .appName('GCS_Spark') \
    .config('spark.jars', '/usr/local/lib/gcs-connector-hadoop3-2.2.7-shaded.jar') \
    .config('spark.hadoop.fs.gs.impl', 'com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem') \
    .getOrCreate()

# save credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/content/.config/application_default_credentials.json'

# Config PySpark to access the GCS
spark._jsc.hadoopConfiguration().set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
spark._jsc.hadoopConfiguration().set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
spark._jsc.hadoopConfiguration().set("fs.gs.auth.service.account.enable", "true")
spark._jsc.hadoopConfiguration().set("google.cloud.auth.service.account.json.keyfile", '/content/.config/application_default_credentials.json')



---


---
3. Set up the source schema and initialize the readStream

In [ ]:
from pyspark.sql.types import *

# create schema
vehicle_schema = StructType([StructField('bearing', IntegerType(), True),
                             StructField('block_id', StringType(), True),
                             StructField('current_status', StringType(), True),
                             StructField('id', StringType(), True),
                             StructField('lat', FloatType(), True),
                             StructField('line_id', StringType(), True),
                             StructField('lon', FloatType(), True),
                             StructField('pattern_id', StringType(), True),
                             StructField('route_id', StringType(), True),
                             StructField('schedule_relationship', StringType(), True),
                             StructField('shift_id', StringType(), True),
                             StructField('speed', FloatType(), True),
                             StructField('stop_id', StringType(), True),
                             StructField('timestamp', TimestampType(), True),
                             StructField('trip_id', StringType(), True)])


#readStreaming
stream = spark.readStream.format("json").schema(vehicle_schema).load("gs://edit-de-project-streaming-data/carris-vehicles")



---


---
4. Write the stream in a bronze layer (landing zone)
* Purpose: Raw data ingestion layer.
* Data Characteristics: Raw, unprocessed, and schema-on-read where feasible.
* Data Storage: Store data exactly as ingested (in this case JSON format).
* Operations: Minimal transformation; only schema enforcement and deduplication.

In [ ]:
df_stops = spark.read.option("header", "true").csv('gs://edit-data-eng-project-group1/LandingZone/GTFS/stops.txt')
df_stops = df_stops.select('stop_id','stop_lat','stop_lon')
df_stops = df_stops.withColumn("stop_lat", df_stops["stop_lat"].cast("float"))
df_stops = df_stops.withColumn("stop_lon", df_stops["stop_lon"].cast("float"))

In [ ]:
#select columns
transform = stream.select('id', 'speed', 'timestamp','line_id','route_id','stop_id','lat', 'lon')
# join tables
transform = transform.join(df_stops, on='stop_id', how='left')

In [ ]:

from pyspark.sql.functions import col, lag,coalesce, current_timestamp, window
from pyspark.sql.window import Window
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from pyspark.sql import functions as F

# watermark is necessarary because of the aggregation
transformed = transform.withWatermark("timestamp", "60 seconds")

windowed_transform = transformed.groupBy("id", "stop_id", F.window("timestamp", "2 minutes")).agg(
    F.first(col("lat")).alias("previous_lat"),
    F.first(col("lon")).alias("previous_lon"),
    F.last(col("lat")).alias("lat"),
    F.last(col("lon")).alias("lon"),
    F.last(col("stop_lat")).alias("stop_lat"),
    F.last(col("stop_lon")).alias("stop_lon")
    )

AnalysisException: [MISSING_AGGREGATION] The non-aggregating expression "lat" is based on columns which are not participating in the GROUP BY clause.
Add the columns or the expression to the GROUP BY, aggregate the expression, or use "any_value(lat)" if you do not care which of the values within a group is returned.;
Project [id#12376, stop_id#12385, window#12611-T60000ms, previous_lat#12623, previous_lon#12625, lat#12627, lon#12629, stop_lat#12631, stop_lon#12633]
+- Project [id#12376, stop_id#12385, window#12611-T60000ms, lat#12377, timestamp#12386-T60000ms, lon#12379, stop_lat#12584, stop_lon#12588, previous_lat#12623, previous_lon#12625, lat#12627, lon#12629, stop_lat#12631, stop_lon#12633, previous_lat#12623, previous_lon#12625, lat#12627, lon#12629, stop_lat#12631, stop_lon#12633]
   +- Window [first(lat#12377, false) windowspecdefinition(id#12376, timestamp#12386-T60000ms ASC NULLS FIRST, specifiedwindowframe(RangeFrame, unboundedpreceding$(), currentrow$())) AS previous_lat#126

In [ ]:
from pyspark.sql.functions import col, sqrt, pow

windowed_transform = windowed_transform.withColumn("distance",
      sqrt(
        pow(col("lat") - col("previous_lat"), 2) + pow(col("lon") - col("previous_lon"), 2)
    )* 110 # each degree is ~100km
                                                   )

windowed_transform = windowed_transform.withColumn("distance_to_stop",      sqrt(
        pow(col("lat") - col("stop_lat"), 2) + pow(col("lon") - col("stop_lon"), 2)
    )* 110 # each degree is ~110km
                                                   )

In [ ]:
agg = windowed_transform.withColumn('speed', col('distance')/(2/60))

agg = agg.withColumn('time_to_stop', (col('distance_to_stop')/col('speed') * 3600))

agg = agg.withColumn(
    'time_to_stop',
    F.from_unixtime(
        F.unix_timestamp(F.lit('00:00:00'), 'HH:mm:ss') + col('time_to_stop'),
        'HH:mm:ss'
    ))

5. Write Stream

In [ ]:
# select folder
folder = 'stream/vehicles6'
gc_folder = 'gs://edit-de-project-streaming-data/datalake/stream/vehicles'


# Output function for each windowed batch
def insert_windowed_vehicles(df, batch_id):
    print(f"Batch ID: {batch_id}")
    df.write.format("parquet").mode("append").save(f"{folder}")


# Write the streaming query with watermark and window
windowed_query = (agg
                  .writeStream
                  .outputMode("append")
                  .foreachBatch(insert_windowed_vehicles)
                  .option('checkpointLocation', f'{folder}/checkpoint')
                  .trigger(processingTime='10 seconds')
                  .start()
)

windowed_query.awaitTermination(30)

False

In [ ]:
windowed_query.isActive

True

In [ ]:
windowed_query.status

{'message': 'Processing new data',
 'isDataAvailable': True,
 'isTriggerActive': True}

In [ ]:
windowed_query.recentProgress

[]

In [ ]:
windowed_query.stop()

4. Transformations

In [ ]:
windowed_query.recentProgress

[]

In [ ]:
# Define the path to the Parquet files
parquet_path = "stream/vehicles5"

# Read the Parquet files into a DataFrame
parquet_df = spark.read.parquet(parquet_path)

# Show the first few rows
parquet_df.show(truncate=False)

# Print the schema to understand the data structure
parquet_df.printSchema()

+--------+-------+------------------------------------------+------------+------------+---------+---------+---------+---------+--------------------+--------------------+------------------+------------+
|id      |stop_id|window                                    |previous_lat|previous_lon|lat      |lon      |stop_lat |stop_lon |distance            |distance_to_stop    |speed             |time_to_stop|
+--------+-------+------------------------------------------+------------+------------+---------+---------+---------+---------+--------------------+--------------------+------------------+------------+
|43|2339 |140349 |{2025-01-22 13:18:00, 2025-01-22 13:20:00}|38.637917   |-9.154437   |38.63827 |-9.154015|38.63906 |-9.152929|0.060684412994725434|0.14763666595197703 |1.8205323898417631|00:04:51    |
|41|1423 |171629 |{2025-01-21 13:42:00, 2025-01-21 13:44:00}|38.904125   |-9.416783   |38.904636|-9.414741|38.9048  |-9.413896|0.2316330739949299  |0.09468030391511913 |6.948992219847898 |00:0